- sections : '정치', '경제', '사회'
- sections = ["pol", "eco","soc"]
- section_ids = ["100", "101","102"]

In [33]:
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime
import warnings
warnings.filterwarnings('ignore')
import re
import pandas as pd
import time
from datetime import datetime
import os
import re
import pymysql

In [34]:
news_df = pd.DataFrame(
    data=None
    , index=None
    , columns=['date','title','content']
)

In [35]:
# ConnectionError방지
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    res = requests.get(url, headers = headers, verify=False)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

In [36]:
# 뉴스의 기본 정보 가져오기
def get_top3_news_info():
    news_urls =[]
    
    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=001"
    
    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)
  
    # 해당 분야 상위 뉴스 3개 가져오기

    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    for li in lis3:
        news_url = li.a.attrs.get('href')
        news_urls.append(news_url)
        
    return news_urls

In [37]:
# 상위 3개 뉴스 크롤링
def F_crawling(news_urls) :
    news_titles = []
    news_contents =[]
    news_dates = []

    for url in news_urls:
        news_html = get_soup_obj(url)

        # 뉴스 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None:
            title = news_html.select_one("#content > div.end_ct > div > h2")

        # 뉴스 본문 가져오기
        content = news_html.select("div#dic_area")
        if content == []:
            content = news_html.select("#articeBody")
            
        # 기사 텍스트만 가져오기
        # list합치기
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern=pattern1, repl='', string=str(title))
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')

        news_titles.append(title)
        news_contents.append(content)

        try:
            html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
            news_date = html_date.attrs['data-date-time']
        except AttributeError:
            news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
            news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
        # 날짜 가져오기
        news_dates.append(news_date)

        # html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
        # news_date = html_date.attrs['data-date-time']
        # news_dates.append(news_date)
    
    a = pd.DataFrame({'date':news_dates,'title':news_titles,'content':news_contents})

    #중복 행 지우기
    a = a.drop_duplicates(keep='first',ignore_index=True)

    return a

In [38]:
def clean_text(d):
  text = re.sub(r'\([^)]*\)', '', str(d))
  text = re.sub(r'\[[^]]*\]', '', text)
  text = re.sub(r'\<[^>]*\>', '', text)
  pattern = r'[^가-힣0-9a-zA-Z\s]'
  text = re.sub(pattern, ' ', text)
  text = re.sub(r'사진', ' ', text)
  text = re.sub(r'.*뉴스', ' ', text)
  text = re.sub("\n", ' ', text)
  text = re.sub("  +", " ", text)
  return text

def text_clean(text):
  pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
  text = re.sub(pattern, '', text)
  pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
  text = re.sub(pattern, '', text)
  pattern = '[a-zA-Z0-9]'    # 숫자와 알파벳 제거
  text = re.sub(pattern, '', text)
  pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
  text = re.sub(pattern, '', text)
  pattern = '<[^>]*>'         # HTML 태그 제거
  text = re.sub(pattern, '', text)
  pattern = '[^\w\s]'         # 특수기호제거
  text = re.sub(pattern, '', text)
  pattern = '[\n]'            # \n제거
  text = re.sub(pattern, '', text)
  pattern = '[\t]'            # \n제거
  text = re.sub(pattern, '', text)
  pattern = '[\']'           
  text = re.sub(pattern, '', text)
  pattern = '[\"]'            
  text = re.sub(pattern, '', text)
  return text  

In [39]:
def to_dbeaver(NewsDate, NewsTitle, NewsContent) :
    conn = pymysql.connect(
        host='34.64.240.96'
        , user='root'
        , password='tndusWkd1.'
        , db='final_project'
        , charset='utf8'
    )
    cur = conn.cursor()
    
    for date,title,content in zip(NewsDate, NewsTitle, NewsContent):
        sql = "INSERT INTO past_news_realtime VALUES ({}, {}, {})".format("\""+date+"\"", "\""+title+"\"", "\""+content+"\"")
        # print(sql)
        try :
            cur.execute(sql)
            conn.commit()
            conn.close()
        except :
            print("에러")
            return
    # conn.commit()
    # conn.close()

In [40]:
# 무한루프 크롤링
while True :
    news_urls = get_top3_news_info()
    a = F_crawling(news_urls)
    df_all = pd.merge(a, news_df, how='outer', indicator=True)
    news_df = pd.concat([news_df, a], ignore_index=True, keys=['date','title','content'])
    news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
    str_expr = '_merge == \"left_only\"'
    df_all2 = df_all.query(str_expr)
    df_all2.drop(columns=['_merge'], inplace=True)

    df_all2.dropna(axis=0, inplace=True)

    df_all2['content'] = df_all2['content'].apply(text_clean)
    df_all2['content'] = df_all2['content'].apply(clean_text)
    df_all2['title'] = df_all2['title'].apply(text_clean)
    df_all2['title'] = df_all2['title'].apply(clean_text)


    df_all2.dropna(axis=0, inplace=True)

    day = datetime.now()
    filepath = 'D:\\big16\\final project\\DATA\\REAL_NEWS_DATA\\real_time_news_{}.csv'.format(day.strftime('%Y%m%d'))
    if not os.path.exists(filepath):
        df_all2.to_csv(filepath, index=False, mode='w', encoding='utf-8-sig')
    else:
        df_all2.to_csv(filepath, index=False, mode='a', encoding='utf-8-sig', header=False) 

    NewsDate = df_all2['date'].to_list()
    NewsTitle = df_all2['title'].to_list()
    NewsContent = df_all2['content'].to_list()

    to_dbeaver(NewsDate, NewsTitle, NewsContent)
                      
    time.sleep(10)